# Dashboard用のdataframeを作る
## Dashboard構想 
### 重賞レース別で以下のグラフを表示するdashboardを作る
* 優勝タイム、2着馬、3着馬、平均タイムの推移をグラフ化(指定で切り替えもできるようにする)  
* 牝馬と牡馬の比率を棒グラフで可視化  
* ジョッキーの比率を棒グラフで可視化  
* 馬の年齢の比率を棒グラフで可視化  

主にrace_dataとhorse_dataの結合をしていく


In [62]:
import pandas as pd
import numpy as np
import pyarrow

In [63]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit              : ba1cccd19da778f0c3a7d6a885685da16a072870
python              : 3.10.12.final.0
python-bits         : 64
OS                  : Darwin
OS-release          : 23.0.0
Version             : Darwin Kernel Version 23.0.0: Fri Sep 15 14:42:57 PDT 2023; root:xnu-10002.1.13~1/RELEASE_ARM64_T8112
machine             : arm64
processor           : arm
byteorder           : little
LC_ALL              : None
LANG                : None
LOCALE              : None.UTF-8

pandas              : 2.1.0
numpy               : 1.24.3
pytz                : 2023.3.post1
dateutil            : 2.8.2
setuptools          : 68.2.2
pip                 : 23.2.1
Cython              : None
pytest              : None
hypothesis          : None
sphinx              : None
blosc               : None
feather             : None
xlsxwriter          : None
lxml.etree          : 4.9.3
html5lib            : 1.1
pymysql             : None
psycopg2            : None
jinja2 

In [66]:
#データの読み込み
#horse_data = pd.read_csv('../data/horse_data_2012_2021.csv',index_col=0)
horse_data = pd.read_parquet('../data/horse_data_2012_2021.parquet')
race_data = pd.read_parquet('../data/race_data_2012_2021.parquet')

# Dashboard用データ向けのhorse_dataの整形

In [67]:
#両方のDataFrameのカラムで重複するところを抽出
drop_target=list(horse_data.columns.intersection(race_data.columns))
#重複するカラムを削除
horse_data=horse_data.drop(columns=drop_target)
#indexからhorse_id列を作る。(先に作ると消えてしまうので後からつける)strとintはmergeできないので型を変換
horse_data['horse_id']=horse_data.index.astype(int)
race_data['race_id']=race_data.index.astype(int)

In [68]:
drop_target

['枠番', '馬番', '人気', '着順', '騎手', '斤量', 'タイム', '着差']

In [69]:
#海外レースのidに英字が含まれてしまうためto_numericで変換(今回海外のレースは使わないため欠損にしてしまって構わない。ただしもとデータには残したいのでこちらのファイルで処理)
horse_data['race_id']=pd.to_numeric(horse_data['race_id'],errors='coerce')

In [70]:
horse_data

,日付,開催,天気,R,レース名,頭数,オッズ,距離,馬場,通過,ペース,上り,馬体重,勝ち馬(2着馬),賞金,race_id,surface,horse_id
2010101358,2014/12/21,4中京6,晴,10.0,桑名特別(500万下),18.0,54.6,1200,重,1-1,35.3-36.5,38.2,468(0),アドマイヤサブリナ,NaN,2.014070e+11,芝,2010101358
2010101358,2014/08/16,2小倉5,曇,8.0,3歳以上500万下,15.0,44.7,1800,稍,1-1-1-2,35.1-37.0,39.2,468(+2),ロードハリケーン,NaN,2.014100e+11,芝,2010101358
2010101358,2014/05/18,3京都8,晴,8.0,4歳以上500万下,18.0,9.4,1400,良,3-4,34.3-35.4,35.3,466(+2),サチノリーダース,NaN,2.014080e+11,芝,2010101358
2010101358,2014/05/03,3京都3,晴,7.0,4歳以上500万下,11.0,36.5,1600,良,1-1,35.4-34.2,34.9,464(-6),ベアトリッツ,75.0,2.014080e+11,芝,2010101358
2010101358,2014/02/23,1小倉6,晴,8.0,4歳以上500万下,18.0,18.0,1200,良,5-8,33.2-34.7,36.3,470(+10),カハラビスティー,NaN,2.014100e+11,芝,2010101358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018106259,2022/05/18,名古屋,晴,5.0,C13組,12.0,1.1,1500,良,1-1-1,0.0-38.7,38.7,476(-7),(ニーニャ),35.0,2.022481e+11,ダ,2018106259
2018106259,2022/01/18,名古屋,曇,3.0,C22組,9.0,1.2,1400,良,1-1-1,0.0-40.1,40.1,483(0),(ノーブルアイランド),32.0,2.022480e+11,ダ,2018106259
2018106259,2021/12/29,笠松,曇,8.0,ビオラ賞,9.0,1.1,1600,稍,1-1-1,0.0-39.3,39.8,483(+1),ハンデンキング,9.0,2.021471e+11,ダ,2018106259
2018106259,2021/12/08,名古屋,晴,5.0,C25組,10.0,1.6,1400,不,1-1-1,0.0-38.8,38.8,482(+18),(ニホンピロビヨンド),32.0,2.021481e+11,ダ,2018106259


# Dashboard用データ向けのrace_dataの整形

In [71]:
race_data

,着順,枠番,馬番,馬名,斤量,騎手,タイム,着差,単勝,人気,調教師,性別,年齢,horse_id,jockey_id,trainer_id,time,体重,増減,race_id
201201010101,1,7,8,アウトシャイン,54.0,池添謙一,1:30.0,None,1.7,1.0,[西] 安田隆行,牝,2,2010101358,1032,438,90.0,442,0,201201010101
201201010101,2,8,10,マイネルブルズアイ,54.0,津村明秀,1:30.4,2.1/2,38.7,8.0,[東] 武藤善則,牡,2,2010101786,1092,1064,90.4,452,-4,201201010101
201201010101,3,8,9,マイネルゴスホーク,54.0,三浦皇成,1:30.5,1/2,7.6,4.0,[東] 古賀慎明,牡,2,2010102157,1122,1079,90.5,478,-8,201201010101
201201010101,4,7,7,ユキカゼ,54.0,荻野琢真,1:30.7,1.1/2,23.9,7.0,[西] 牧田和弥,牡,2,2010103377,1112,1123,90.7,494,12,201201010101
201201010101,5,2,2,スピーディシャープ,54.0,小林徹弥,1:30.8,クビ,136.7,10.0,[西] 目野哲也,牡,2,2010104671,722,387,90.8,414,0,201201010101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202109021212,12,8,15,オーバーディリバー,55.0,横山武史,1:25.2,1,5.5,3.0,[東] 萩原清,牡,5,2016100182,1170,1017,85.2,482,0,202109021212
202109021212,13,7,13,ハルサカエ,52.0,坂井瑠星,1:25.3,1/2,175.7,16.0,[東] 大竹正博,牝,5,2016103079,1163,1102,85.3,512,2,202109021212
202109021212,14,5,9,アプルーヴァル,54.0,和田竜二,1:25.6,1.3/4,21.6,10.0,[西] 今野貞一,牡,6,2015104950,1018,1128,85.6,512,0,202109021212
202109021212,15,2,3,カバジェーロ,54.0,国分恭介,1:25.8,1,74.4,12.0,[西] 橋口慎介,牡,4,2017101830,1124,1154,85.8,494,0,202109021212


### データの結合

In [72]:
df_join =pd.merge(race_data,horse_data,on=['race_id','horse_id'],how='left')

In [73]:
df_join.columns

Index(['着順', '枠番', '馬番', '馬名', '斤量', '騎手', 'タイム', '着差', '単勝', '人気', '調教師',
       '性別', '年齢', 'horse_id', 'jockey_id', 'trainer_id', 'time', '体重', '増減',
       'race_id', '日付', '開催', '天気', 'R', 'レース名', '頭数', 'オッズ', '距離', '馬場', '通過',
       'ペース', '上り', '馬体重', '勝ち馬(2着馬)', '賞金', 'surface'],
      dtype='object')

In [74]:
#columの並び替え
#target_colsに入れた順で後ろに入っていく
target_cols = ['馬名','horse_id','レース名','race_id']
for target_col in target_cols:
    #対象列を一時的に保持
    df_target = df_join[target_col]
    #対象列を元のDataFrameから削除
    df_join.drop(columns=[target_col],inplace=True)
    #対象列を目的の場所に挿入
    df_join.insert(0,target_col,df_target)



In [75]:
#描画の時扱いやすいように開催年カラムを作る
df_join['開催年']=df_join['日付'].str[:4]

In [76]:
df_join.rename(columns={'distance':'距離'},inplace=True)
df_join.dropna(subset=['開催'],inplace=True)
#開催地でも絞れる様に(たとえば改修などで別のところでやっていたところは省ける様にしたいため）開催地を抜き出す
df_join['開催地']=df_join['開催'].str[1:3]
#G1,G2,G3を楽に確認できる様race_rank列を作る
df_join['race_rank'] = df_join['レース名'].str[-3:-1]

In [77]:
#race_rank列が(G1,G2,G3)のみのものに絞りdashboard用データとする。
dashboard_data = df_join.query('race_rank in ["G1","G2","G3"]')

In [78]:
#dashboard_data.to_csv('dashboard_data.csv',header=True)

In [79]:
#parquetファイルに保存
dashboard_data.to_parquet('dashboard_data.parquet')